In [3]:
import axelrod as axl
import networkx as nx
import matplotlib.pyplot as plt
from scipy.io import mmread
import numpy as np
import pandas as pd

Base Algorithm

In [ ]:
file_path = "fb_graph/matname.mtx"
sparse_matrix = mmread(file_path)
graph = nx.Graph(sparse_matrix)

strategies = [
    axl.Cooperator(),
    axl.Defector(),
    axl.TitForTat(),
    axl.Grudger(),
    axl.Random(),
    axl.ZDExtortion(),
    axl.Capri(),
    axl.Appeaser()
]

players = []
for node in graph.nodes:
    players.append(np.random.choice(strategies))

# turns is ther number of iterations between each pair
spatial_tournament = axl.Tournament(
    players, edges=graph.edges, turns=100, repetitions=1)
results = spatial_tournament.play(filename="test.csv")

df = pd.read_csv("test.csv")


In [5]:
df = pd.read_csv("100_turns.csv")

print(df.columns)

Index(['Interaction index', 'Player index', 'Opponent index', 'Repetition',
       'Player name', 'Opponent name', 'Actions', 'Score', 'Score difference',
       'Turns', 'Score per turn', 'Score difference per turn', 'Win',
       'Initial cooperation', 'Cooperation count', 'CC count', 'CD count',
       'DC count', 'DD count', 'CC to C count', 'CC to D count',
       'CD to C count', 'CD to D count', 'DC to C count', 'DC to D count',
       'DD to C count', 'DD to D count', 'Good partner'],
      dtype='object')


Getting best strageties for X turns.

In [11]:
import pandas as pd

# Load results from the tournament
df = pd.read_csv("100_turns.csv")

# Compute total score for each player
scores = df.groupby("Player index")["Score"].sum()

# Extract unique strategies based on the "Player index" and "Player name"
strategy_mapping = df[["Player index", "Player name"]
                      ].drop_duplicates().set_index("Player index")["Player name"]

# Add strategy information to the scores DataFrame
scores_with_strategy = pd.DataFrame(scores).reset_index()
scores_with_strategy["Strategy"] = scores_with_strategy["Player index"].map(
    strategy_mapping)

# Compute mean score by strategy
strategy_scores = scores_with_strategy.groupby("Strategy")["Score"].mean()

# Calculate the overall mean score
overall_mean_score = strategy_scores.mean()

# Categorize strategies as good or bad based on their mean score
good_strategies = strategy_scores[strategy_scores > overall_mean_score]
bad_strategies = strategy_scores[strategy_scores <= overall_mean_score]

# Output the results
print("Good Strategies:")
for strategy, mean_score in good_strategies.items():
    print(f"Strategy: {strategy} - Mean Score: {mean_score:.2f}")

print("\nBad Strategies:")
for strategy, mean_score in bad_strategies.items():
    print(f"Strategy: {strategy} - Mean Score: {mean_score:.2f}")

Good Strategies:
Strategy: Appeaser - Mean Score: 15469.33
Strategy: CAPRI - Mean Score: 16736.94
Strategy: Cooperator - Mean Score: 15437.00
Strategy: Grudger - Mean Score: 18450.86
Strategy: Tit For Tat - Mean Score: 16543.53

Bad Strategies:
Strategy: Defector - Mean Score: 14075.96
Strategy: Random: 0.5 - Mean Score: 11875.78
Strategy: ZD-Extortion: 0.2, 0.1, 1 - Mean Score: 12092.30


Checks what is better, playing as Defector or Cooperator

In [14]:
import pandas as pd

# Load results from the tournament
df = pd.read_csv("100_turns.csv")

# Separate games based on the player's strategy
cooperator_games = df[df["Player name"] == "Cooperator"]
defector_games = df[df["Player name"] == "Defector"]

# Calculate mean and standard deviation for Cooperators
cooperator_avg_score = cooperator_games["Score"].mean()
cooperator_std_dev = cooperator_games["Score"].std()

# Calculate mean and standard deviation for Defectors
defector_avg_score = defector_games["Score"].mean()
defector_std_dev = defector_games["Score"].std()

# Analyze performance based on opponent's strategy
cooperator_vs_opponents = cooperator_games.groupby(
    "Opponent name")["Score"].agg(["mean", "std"])
defector_vs_opponents = defector_games.groupby(
    "Opponent name")["Score"].agg(["mean", "std"])

# Output the results
print(f"Average Score for Cooperators: {
      cooperator_avg_score:.2f} (Std Dev: {cooperator_std_dev:.2f})")
print(f"Average Score for Defectors: {
      defector_avg_score:.2f} (Std Dev: {defector_std_dev:.2f})")

if cooperator_avg_score > defector_avg_score:
    print("It is better to play as a Cooperator.")
elif defector_avg_score > cooperator_avg_score:
    print("It is better to play as a Defector.")
else:
    print("Playing as a Cooperator or Defector is equally effective.")

print("\nPerformance of Cooperators Against Opponents:")
print(cooperator_vs_opponents)

print("\nPerformance of Defectors Against Opponents:")
print(defector_vs_opponents)

Average Score for Cooperators: 222.88 (Std Dev: 107.67)
Average Score for Defectors: 202.53 (Std Dev: 141.99)
It is better to play as a Cooperator.

Performance of Cooperators Against Opponents:
                                 mean        std
Opponent name                                   
Appeaser                   300.000000   0.000000
CAPRI                      300.000000   0.000000
Cooperator                 300.000000   0.000000
Defector                     0.000000   0.000000
Grudger                    300.000000   0.000000
Random: 0.5                150.228571  15.106975
Tit For Tat                300.000000   0.000000
ZD-Extortion: 0.2, 0.1, 1  134.112330  21.726942

Performance of Defectors Against Opponents:
                                 mean        std
Opponent name                                   
Appeaser                   300.000000   0.000000
CAPRI                      104.000000   0.000000
Cooperator                 500.000000   0.000000
Defector                 

In [15]:
import pandas as pd
import networkx as nx
from scipy.io import mmread

# Load results from the tournament
df = pd.read_csv("100_turns.csv")

# Load the graph to compute clustering coefficients
file_path = "fb_graph/matname.mtx"  # Path to your graph file
sparse_matrix = mmread(file_path)
graph = nx.Graph(sparse_matrix)

# Compute clustering coefficients for all nodes
clustering_coeffs = nx.clustering(graph)
clustering_df = pd.DataFrame(list(clustering_coeffs.items()), columns=[
                             "Player index", "Clustering Coefficient"])

# Merge clustering coefficients with player scores
scores = df.groupby("Player index")["Score"].sum()
strategy_mapping = df[["Player index", "Player name"]
                      ].drop_duplicates().set_index("Player index")["Player name"]
scores_with_clustering = pd.DataFrame(scores).reset_index()
scores_with_clustering["Strategy"] = scores_with_clustering["Player index"].map(
    strategy_mapping)
scores_with_clustering = scores_with_clustering.merge(
    clustering_df, on="Player index")

# Calculate average score and standard deviation for each strategy
strategy_stats = scores_with_clustering.groupby(
    "Strategy")["Score"].agg(["mean", "std"]).reset_index()

# Find strategies with highest/lowest average score standard deviation
highest_std_strategy = strategy_stats.loc[strategy_stats["std"].idxmax()]
lowest_std_strategy = strategy_stats.loc[strategy_stats["std"].idxmin()]

# Categorize nodes into high and low clustering coefficient groups
median_clustering = scores_with_clustering["Clustering Coefficient"].median()
high_cluster_nodes = scores_with_clustering[scores_with_clustering["Clustering Coefficient"] > median_clustering]
low_cluster_nodes = scores_with_clustering[scores_with_clustering["Clustering Coefficient"]
                                           <= median_clustering]

# Calculate average score for strategies among high-cluster-coefficient nodes
high_cluster_stats = high_cluster_nodes.groupby(
    "Strategy")["Score"].agg(["mean", "std"]).reset_index()
highest_mean_high_cluster = high_cluster_stats.loc[high_cluster_stats["mean"].idxmax(
)]
lowest_mean_high_cluster = high_cluster_stats.loc[high_cluster_stats["mean"].idxmin(
)]

# Calculate average score standard deviation for strategies among low-cluster-coefficient nodes
low_cluster_stats = low_cluster_nodes.groupby(
    "Strategy")["Score"].agg(["mean", "std"]).reset_index()
highest_std_low_cluster = low_cluster_stats.loc[low_cluster_stats["std"].idxmax(
)]
lowest_std_low_cluster = low_cluster_stats.loc[low_cluster_stats["std"].idxmin(
)]

# Output results
print("Highest Average Score Standard Deviation:")
print(f"Strategy: {
      highest_std_strategy['Strategy']} - Std Dev: {highest_std_strategy['std']:.2f}")

print("\nLowest Average Score Standard Deviation:")
print(f"Strategy: {lowest_std_strategy['Strategy']
                   } - Std Dev: {lowest_std_strategy['std']:.2f}")

print("\nHighest Average Score Among High Cluster-Coefficient Nodes:")
print(f"Strategy: {highest_mean_high_cluster['Strategy']
                   } - Mean Score: {highest_mean_high_cluster['mean']:.2f}")

print("\nLowest Average Score Among High Cluster-Coefficient Nodes:")
print(f"Strategy: {lowest_mean_high_cluster['Strategy']
                   } - Mean Score: {lowest_mean_high_cluster['mean']:.2f}")

print("\nHighest Average Score Standard Deviation Among Low Cluster-Coefficient Nodes:")
print(f"Strategy: {highest_std_low_cluster['Strategy']
                   } - Std Dev: {highest_std_low_cluster['std']:.2f}")

print("\nLowest Average Score Standard Deviation Among Low Cluster-Coefficient Nodes:")
print(f"Strategy: {lowest_std_low_cluster['Strategy']
                   } - Std Dev: {lowest_std_low_cluster['std']:.2f}")

Highest Average Score Standard Deviation:
Strategy: Tit For Tat - Std Dev: 16274.91

Lowest Average Score Standard Deviation:
Strategy: Random: 0.5 - Std Dev: 10267.23

Highest Average Score Among High Cluster-Coefficient Nodes:
Strategy: Grudger - Mean Score: 13814.28

Lowest Average Score Among High Cluster-Coefficient Nodes:
Strategy: Random: 0.5 - Mean Score: 8712.89

Highest Average Score Standard Deviation Among Low Cluster-Coefficient Nodes:
Strategy: Tit For Tat - Std Dev: 19893.07

Lowest Average Score Standard Deviation Among Low Cluster-Coefficient Nodes:
Strategy: Random: 0.5 - Std Dev: 11700.84
